In [1]:
import logging
import requests
import asyncio
import websockets
from epdlib import Screen, Update
from epdlib import Layout

In [4]:
logging.basicConfig(level=logging.WARNING)
logging.debug('foobar')

# PaperPi Structure
Supervisor will loop and poll each plugin for an update. Plugins are all`Plugin()` objects with a `poll` method

When polled `Plugin()` objects respond with a structured list (tuple/dict) with the following information:
* Priority(`int`): 0 (high) -- 10+ (low); plugins with negative values are excluded
* Update Required(`bool`): True update; False do not update

Plugin objects require the following parameters:
* configuration(`dict`) argparse/config.ini style dict
* resolution (this may be pulled from the configuration?)
* name(`str`) human readable name for logging and reference
* update_function(`func`): function that will run and provide updates and status

In [5]:
def strict_enforce(*types):
    """strictly enforce type compliance within classes
    
    Usage:
        @strict_enforce(type1, type2, (type3, type4))
        def foo(val1, val2, val3):
            ...
    """
    def decorator(f):
        def new_f(self, *args, **kwds):
            #we need to convert args into something mutable   
            newargs = []        
            for (a, t) in zip(args, types):
                if not isinstance(a, t):
                    raise TypeError(f'"{a}" is not type {t}')
#                 newargs.append( t(a)) #feel free to have more elaborated convertion
            return f(self, *args, **kwds)
        return new_f
    return decorator

In [6]:
def add_method(cls):
    def decorator(func):
        @wraps(func) 
        def wrapper(self, *args, **kwargs): 
            return func(*args, **kwargs)
        setattr(cls, func.__name__, wrapper)
        # Note we are not binding func, but wrapper which accepts self but does exactly the same as func
        return func # returning func means func can still be used normally
    return decorator

In [42]:
class Plugin:
    # plugin attributes:
    # [x] name
    # [x] last time updated
    # [x] Layout
    # [x] Image
    # [x] resolution
    # [x] priority for inclusion in display loop -- maximum priority
    # [ ] configuration -- argParse style dict of main program configuration
    
    # Plugin methods:
    # * check_for_update
    # [x] update
    def __init__(self, name=None, resolution=None, layout=None, 
                 max_priority=-1, update_function=None, config={}):
        self.timer = Update()
        self.resolution = resolution
        self.max_priority = max_priority
        self.config = config
        if resolution:
            self.layout = layout
        self.add_update_function(update_function)
    
    
    def add_update_function(self, function):
        '''add an update_function to the namespace of this instance of object
            all update_functions must accept at a minimum the following kwargs:
                * config(`dict`) configuration dictionary passed to all update functions
            all update_functions must return a tuple of:
                (update, data, priority)
                
                update(`bool`): True, data has been updated; display requires update
                data(`dict`): structured for a Layout().update 
                priority(`int` or `float`): priority for inclusion in display loop
                    positive values that approach 0 have higher priority

        Args:
            fuction(`function`): function to add as self.update_function
            
        see https://stackoverflow.com/a/2982/5530152 regarding descriptor protocol''' 
        if function:
            self.update_function = function.__get__(self)
        else:
            self.update_function = self._null_function.__get__(self)
    
    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, name):
        self._name = str(name)
        

    @property
    def last_update(self):
        '''display the monotnic time since this module was updated'''
        return self.timer.last_updated
    
    @property
    def max_priority(self):
        '''maximum priority for displaying this module in the update loop
            positive values that approach zero are higher priority; zero is higest
            negative values are excluded entirely
        
        max_priority(`int` or `float`)
        '''
        return self._max_priority
    
    @max_priority.setter
    @strict_enforce(int, float)
    def max_priority(self, max_priority):
        self._max_priority = max_priority
    
    @property
    def image(self):
        '''constructed image based on the layout'''
        return self.layout_obj.concat()
    
    @property
    def resolution(self):
        '''width x height of screen
        resolution(`tuple` of `int`): (W x H)'''
        return self._resolution
    
    @resolution.setter
    def resolution(self, resolution):
        if isinstance(resolution, (type(None), list, tuple)):
            self._resolution = resolution
            if hasattr(self, 'layout_obj'):
                # trigger recalculation of the layout if the resolution changes
                layout = self.layout
                self.layout = layout
        else:
            raise ValueError('`resolution` must be a list-like object')
            
    
    @property
    def layout(self):
        '''layout dictionary for placing text and images blocks on the screen'''
        return self.layout_obj.layout
    
    @layout.setter
    def layout(self, layout):
        if self.resolution:
            self.layout_obj = Layout(resolution=self.resolution, layout=layout) 
        else:
            raise ValueError(f'cannot set `layout` without a valid resolution: {self.resolution}')
    
        

#     def update(self, *args, **kwargs):
#         '''update the monotonic clock of the object and call the update_function
        
#         Args:
#             *args
#             **kwargs
        
#         Returns:
#             `bool` True if updated, False if no update  
#         '''

#         update, data, priority = self.update_function(*args, **kwargs)
        
#         if update:
#             self.layout_obj.update_contents(data)
#             self.priority = priority
#             self.timer.update()
#             return True
#         else:
#             return False
        

        
    def _null_function(self, *args, **kwargs):
        logging.info('default placeholder function')
        return (False, {}, -1)
    
    def poll(self, *args, **kwargs):
        '''call the update_function and check if there is a needed update
        
        Args:
            *args
            **kwargs
        
        Returns:
            `bool` True if updated, False if no update  
        '''

        update, data, priority = self.update_function(*args, **kwargs)
        
        if update:
            self.layout_obj.update_contents(data)
            self.priority = priority
            self.timer.update()
            return True
        else:
            return False
        



    
#     @property
#     def update_func(self, args, kwargs):
#         '''update function for this object
        
#         Args:
#             func(`function`): function that returns a tuple of (`bool`, `dict`) 
#                 where bool is update status (true updates, false passes)
#                 and dict is values to use in the update'''
#         val = self._update_func
#         self.update(*val)
#         return val
    
#     @update_func.setter
#     def update_func(self, func):
#         self._update_func = func    

In [43]:
plugin_digit_clock = Plugin(name='digit clock', 
                            resolution=(500, 350), 
                            layout=clock_layout, 
                            update_function=clock_plugin,
                            config={'update_interval': 10})

In [44]:
plugin_digit_clock.update_function()

{'update_interval': 10}
0.7354078999778721


(False, {}, 0)

In [65]:
plugin_digit_clock.timer.last_updated

4.603553285007365

In [64]:
plugin_digit_clock.poll()

{'update_interval': 10}
43.803485823009396


True

In [67]:
clock_layout = {
    'digit_time': {
        'image': None,
        'max_lines': 3,
        'width': 1,
        'height': 1,
        'abs_coordinates': (0, 0),
        'rand': True,
        'font': '../fonts/Dosis/Dosis-VariableFontwght.ttf',
    },
}

class A:
    pass

test = A()

test.config = {'update_interval': 10}
test.timer = Update()
test.max_priority = 0

In [69]:
from datetime import datetime
def clock_plugin(self):
    if self.timer.last_updated > self.config['update_interval']:
        update = True
        data = {'digit_time': datetime.now().strftime("%H:%M:%S")}
        priority = self.max_priority
    else:
        update = False
        data = {}
        priority = self.max_priority + 1

    return (update, data, priority)
    

In [ ]:
my_time = datetime.now().strftime("%H:%M:%S")

In [ ]:
my_time

In [ ]:
r = requests.post('http://localhost:24879/player/current')

In [ ]:
r = requests.('http://localhost:24879/events')

In [ ]:
r.request

In [ ]:
def spotify(plugin, host='localhost', port=24879, path='/player/current'):
    if not isinstance(plugin, Plugin):
        raise TypeError('`plugin` must be of type Plugin')
        
    url = f"http://{host}:{port}/{path}"
    
    
    return plugin

In [ ]:
s_plugin = Plugin(name=spotify)
s_plugin.last_update

In [ ]:
layout = {
        'title': {                       # text only block
            'image': None,               # do not expect an image
            'max_lines': 2,              # number of lines of text
            'width': 1,                  # 1/1 of the width - this stretches the entire width of the display
            'height': 2/3,               # 1/3 of the entire height
            'abs_coordinates': (0, 0),   # this block is the key block that all other blocks will be defined in terms of
            'hcenter': True,             # horizontally center text
            'vcenter': True,             # vertically center text 
            'relative': False,           # this block is not relative to any other. It has an ABSOLUTE position (0, 0)
            'font': '../fonts/Anton/Anton-Regular.ttf', # path to font file
            'font_size': None            # Calculate the font size because none was provided
        },
    
        'artist': {
            'image': None,
            'max_lines': 1,
            'width': 1,
            'height': 1/3,
            'abs_coordinates': (0, None),   # X = 0, Y will be calculated
            'hcenter': True,
            'vcenter': True,
            'font': '../fonts/Dosis/Dosis-VariableFontwght.ttf',
            'relative': ['artist', 'title'], # use the X postion from abs_coord from `artist` (this block: 0)
                                           # calculate the y position based on the size of `title` block
        }
}